In [1]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

In [2]:
df = pd.read_excel('DEA Formulation.xlsx', header=0)
print(df)

   Branch  Loans  Deposits  Expense  FTE
0       1     10        31      100    5
1       2     15        25      100    5
2       3     20        30      100    5
3       4     23        23      100    6
4       5     30        20      100    5


In [3]:
def efficiency_score(branch):

    L=branch[1]
    D=branch[2]
    E=branch[3]
    F=branch[4]

    oD = LpVariable(name="oD", lowBound=0)
    oL = LpVariable(name="oL", lowBound=0)
    iE = LpVariable(name="iE", lowBound=0)
    iF = LpVariable(name="iF", lowBound=0)

    # Add the constraints to the model
    model = LpProblem(name="DEA_Formulation", sense=LpMaximize)
    for index, row in df.iterrows():
        model +=  (row[1]*oL+row[2]*oD-row[3]*iE-row[4]*iF <=0, 'constraint.{i}'.format(i=index))
    model += ((E*iE+F*iF) == 1, "input_scaling")
    # Add the objective function to the model
    obj_func = L*oL+D*oD
    model += obj_func

    # Solve the problem
    model.solve()

    # print(pulp.LpStatus[model.status]) # See if the model has been optimized

    for variable in model.variables():
        # print('{name}={value}'.format(name=variable.name, value=variable.varValue)) # print the optimal values of all variables
        # find optimal var values used to calculate optimized objective function value.
        if variable.name=='oL':
            oLo=variable.varValue
        if variable.name=='oD':
            oDo=variable.varValue
    return L*oLo+D*oDo # return the value of the objective function

#branch= pd.array(df.loc[1])
#print('Branch:',branch)
#efficiency_score(branch)    #get the efficiency score for a particular branch

result=df.apply(efficiency_score, axis=1)
print(result)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var/folders/nk/k6fq4zkd6lxfpjmy32h3lk0w0000gn/T/65dfccb4e7b6428d9f878873d73e89e5-pulp.lp
 -o /var/folders/nk/k6fq4zkd6lxfpjmy32h3lk0w0000gn/T/65dfccb4e7b6428d9f878873d73e89e5-pulp.sol
Reading problem data from '/var/folders/nk/k6fq4zkd6lxfpjmy32h3lk0w0000gn/T/65dfccb4e7b6428d9f878873d73e89e5-pulp.lp'...
6 rows, 4 columns, 22 non-zeros
11 lines were read
GLPK Simplex Optimizer 5.0
6 rows, 4 columns, 22 non-zeros
Preprocessing...
6 rows, 4 columns, 22 non-zeros
Scaling...
 A: min|aij| =  5.000e+00  max|aij| =  1.000e+02  ratio =  2.000e+01
GM: min|aij| =  6.810e-01  max|aij| =  1.468e+00  ratio =  2.156e+00
EQ: min|aij| =  4.637e-01  max|aij| =  1.000e+00  ratio =  2.156e+00
Constructing initial basis...
Size of triangular part is 6
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (2)
*     1: obj =   1.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0